In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)
pd.set_option('display.max_row',None)
pd.set_option('display.expand_frame_repr',False)
pd.set_option('display.max_colwidth',None)

In [2]:
deliveries = pd.read_csv('remaining_ballbyball.csv')
matches = pd.read_csv('remaining_match.csv')

In [15]:
mdf=matches

In [3]:
df=deliveries

In [4]:
df.head(1)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,bowler,non_striker,runs_of_bat,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,1359475,1,Chennai Super Kings,Gujarat Titans,0,1,DP Conway,Mohammed Shami,RD Gaikwad,0,0,0,NaN,0,NaN,NaN,NaN


In [9]:
#df = df.rename(columns={'wicket_type':'dismissal_kind'})

In [11]:
matches.rename(columns = {'id':'match_id'}, inplace = True)

In [17]:
mdf.head(1)

,match_id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,1359475,2023,Ahmedabad,2023-03-31,League,Rashid Khan,"Narendra Modi Stadium, Ahmedabad",Chennai Super Kings,Gujarat Titans,Gujarat Titans,field,Gujarat Titans,wickets,5.0,179.0,20.0,N,NaN,Nitin Menon,HAS Khalid


In [19]:
comb=pd.merge(df,mdf,on = 'match_id',how ='left')

In [21]:
comb.tail(1)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,bowler,non_striker,runs_of_bat,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
34965,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,3,VR Iyer,Shahbaz Ahmed,SS Iyer,1,0,1,NaN,0,NaN,NaN,NaN,2024,Chennai,2024-05-26,Final,MA Starc,"MA Chidambaram Stadium, Chepauk, Chennai",Sunrisers Hyderabad,Kolkata Knight Riders,Sunrisers Hyderabad,bat,Kolkata Knight Riders,wickets,8.0,114.0,20.0,N,NaN,J Madanagopal,Nitin Menon


In [23]:
comb.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'bowler', 'non_striker', 'runs_of_bat', 'extra_runs',
       'total_runs', 'extras_type', 'is_wicket', 'player_dismissed',
       'dismissal_kind', 'fielder', 'season', 'city', 'date', 'match_type',
       'player_of_match', 'venue', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'winner', 'result', 'result_margin', 'target_runs',
       'target_overs', 'super_over', 'method', 'umpire1', 'umpire2'],
      dtype='object')

In [25]:
def wintoss_winmatch(toss_winner,winner):
    if toss_winner == winner:
        return 1
    else:
        return 0

In [27]:
mdf['wintoss_winmatch'] = mdf.apply(lambda x: wintoss_winmatch(x['toss_winner'],x['winner']),axis = 1)

In [33]:
mdf[['winner','toss_winner','wintoss_winmatch']].tail(10)

,winner,toss_winner,wintoss_winmatch
135,Royal Challengers Bengaluru,Delhi Capitals,0
136,Delhi Capitals,Lucknow Super Giants,0
137,Punjab Kings,Rajasthan Royals,0
138,Lucknow Super Giants,Mumbai Indians,0
139,Royal Challengers Bengaluru,Chennai Super Kings,0
140,Sunrisers Hyderabad,Punjab Kings,0
141,Kolkata Knight Riders,Sunrisers Hyderabad,0
142,Rajasthan Royals,Rajasthan Royals,1
143,Sunrisers Hyderabad,Rajasthan Royals,0
144,Kolkata Knight Riders,Sunrisers Hyderabad,0


In [31]:
100*len(mdf[mdf.wintoss_winmatch == 1])/len(mdf)

44.827586206896555

In [35]:
len(mdf)

145

In [37]:
pd.DataFrame(mdf.groupby('season')['wintoss_winmatch'].sum()/mdf.groupby('season')['wintoss_winmatch'].count())

,wintoss_winmatch
season,
2023,0.459459
2024,0.436620


In [41]:
print(mdf.columns)


Index(['match_id', 'season', 'city', 'date', 'match_type', 'player_of_match',
       'venue', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner',
       'result', 'result_margin', 'target_runs', 'target_overs', 'super_over',
       'method', 'umpire1', 'umpire2', 'wintoss_winmatch'],
      dtype='object')


In [65]:
team_name = "Kolkata Knight Riders"

# Filter matches where KKR was either team1 or team2
kkr_matches = mdf[(mdf['team1'] == team_name) | (mdf['team2'] == team_name)].copy()

# Ensure the date column is in datetime format
kkr_matches['date'] = pd.to_datetime(kkr_matches['date'])

# Identify the opponent team
kkr_matches['opponent'] = kkr_matches.apply(lambda x: x['team1'] if x['team2'] == team_name else x['team2'], axis=1)

# Group by opponent, venue, date, toss winner, and match winner
kkr_analysis = kkr_matches.groupby(['opponent', 'venue', 'date', 'toss_winner', 'winner']).size().reset_index(name='matches_played')

# Sort by date in ascending order
kkr_analysis = kkr_analysis.sort_values(by='date')

# Display the result
kkr_analysis


,opponent,venue,date,toss_winner,winner,matches_played
16,Punjab Kings,"Punjab Cricket Association IS Bindra Stadium, Mohali, Chandigarh",2023-04-01,Kolkata Knight Riders,Punjab Kings,1
19,Royal Challengers Bangalore,"Eden Gardens, Kolkata",2023-04-06,Royal Challengers Bangalore,Kolkata Knight Riders,1
7,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",2023-04-09,Gujarat Titans,Kolkata Knight Riders,1
23,Sunrisers Hyderabad,"Eden Gardens, Kolkata",2023-04-14,Kolkata Knight Riders,Sunrisers Hyderabad,1
12,Mumbai Indians,"Wankhede Stadium, Mumbai",2023-04-16,Mumbai Indians,Mumbai Indians,1
3,Delhi Capitals,"Arun Jaitley Stadium, Delhi",2023-04-20,Delhi Capitals,Delhi Capitals,1
0,Chennai Super Kings,"Eden Gardens, Kolkata",2023-04-23,Kolkata Knight Riders,Chennai Super Kings,1
20,Royal Challengers Bangalore,"M Chinnaswamy Stadium, Bengaluru",2023-04-26,Royal Challengers Bangalore,Kolkata Knight Riders,1
6,Gujarat Titans,"Eden Gardens, Kolkata",2023-04-29,Gujarat Titans,Gujarat Titans,1
27,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal, Hyderabad",2023-05-04,Kolkata Knight Riders,Kolkata Knight Riders,1


In [75]:
team_name = "Kolkata Knight Riders"

# Filter matches where KKR was either team1 or team2
kkr_matches = mdf[(mdf['team1'] == team_name) | (mdf['team2'] == team_name)].copy()

# Ensure the date column is in datetime format
kkr_matches['date'] = pd.to_datetime(kkr_matches['date'])

# Identify the opponent team
kkr_matches['opponent'] = kkr_matches.apply(lambda x: x['team1'] if x['team2'] == team_name else x['team2'], axis=1)

# Group by opponent, venue, date, toss winner, toss decision, match winner, result margin, and target runs
kkr_analysis = kkr_matches.groupby(
    ['opponent', 'venue', 'date', 'toss_winner', 'toss_decision', 'winner', 'result_margin', 'target_runs']
).size().reset_index(name='matches_played')

# Sort by date in ascending order
kkr_analysis = kkr_analysis.sort_values(by='date')

# Display the result
kkr_analysis


,opponent,venue,date,toss_winner,toss_decision,winner,result_margin,target_runs,matches_played
16,Punjab Kings,"Punjab Cricket Association IS Bindra Stadium, Mohali, Chandigarh",2023-04-01,Kolkata Knight Riders,field,Punjab Kings,7.0,154.0,1
19,Royal Challengers Bangalore,"Eden Gardens, Kolkata",2023-04-06,Royal Challengers Bangalore,field,Kolkata Knight Riders,81.0,205.0,1
7,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",2023-04-09,Gujarat Titans,bat,Kolkata Knight Riders,3.0,205.0,1
23,Sunrisers Hyderabad,"Eden Gardens, Kolkata",2023-04-14,Kolkata Knight Riders,field,Sunrisers Hyderabad,23.0,229.0,1
12,Mumbai Indians,"Wankhede Stadium, Mumbai",2023-04-16,Mumbai Indians,field,Mumbai Indians,5.0,186.0,1
3,Delhi Capitals,"Arun Jaitley Stadium, Delhi",2023-04-20,Delhi Capitals,field,Delhi Capitals,4.0,128.0,1
0,Chennai Super Kings,"Eden Gardens, Kolkata",2023-04-23,Kolkata Knight Riders,field,Chennai Super Kings,49.0,236.0,1
20,Royal Challengers Bangalore,"M Chinnaswamy Stadium, Bengaluru",2023-04-26,Royal Challengers Bangalore,field,Kolkata Knight Riders,21.0,201.0,1
6,Gujarat Titans,"Eden Gardens, Kolkata",2023-04-29,Gujarat Titans,field,Gujarat Titans,7.0,180.0,1
27,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal, Hyderabad",2023-05-04,Kolkata Knight Riders,bat,Kolkata Knight Riders,5.0,172.0,1


In [73]:
team_name = "Kolkata Knight Riders"
home_venue = "Eden Gardens, Kolkata"

# Filter matches where KKR was either team1 or team2
kkr_matches = mdf[(mdf['team1'] == team_name) | (mdf['team2'] == team_name)].copy()

# Ensure the date column is in datetime format
kkr_matches['date'] = pd.to_datetime(kkr_matches['date'])

# Identify the opponent team
kkr_matches['opponent'] = kkr_matches.apply(lambda x: x['team1'] if x['team2'] == team_name else x['team2'], axis=1)

# Categorize matches as Home or Away
kkr_matches['match_type'] = kkr_matches['venue'].apply(lambda x: 'Home' if x == home_venue else 'Away')

# Filter matches where KKR won the toss
kkr_toss_wins = kkr_matches[kkr_matches['toss_winner'] == team_name]

# Group by match type, opponent, venue, date, toss decision, toss winner, and match winner
kkr_toss_analysis = kkr_toss_wins.groupby(['match_type', 'opponent', 'venue', 'date', 'toss_decision', 'toss_winner', 'winner']).size().reset_index(name='matches_played')

# Sort by date
kkr_toss_analysis = kkr_toss_analysis.sort_values(by='date')

# Display the result
kkr_toss_analysis


,match_type,opponent,venue,date,toss_decision,toss_winner,winner,matches_played
1,Away,Punjab Kings,"Punjab Cricket Association IS Bindra Stadium, Mohali, Chandigarh",2023-04-01,field,Kolkata Knight Riders,Punjab Kings,1
7,Home,Sunrisers Hyderabad,"Eden Gardens, Kolkata",2023-04-14,field,Kolkata Knight Riders,Sunrisers Hyderabad,1
4,Home,Chennai Super Kings,"Eden Gardens, Kolkata",2023-04-23,field,Kolkata Knight Riders,Chennai Super Kings,1
3,Away,Sunrisers Hyderabad,"Rajiv Gandhi International Stadium, Uppal, Hyderabad",2023-05-04,bat,Kolkata Knight Riders,Kolkata Knight Riders,1
5,Home,Lucknow Super Giants,"Eden Gardens, Kolkata",2023-05-20,field,Kolkata Knight Riders,Lucknow Super Giants,1
2,Away,Royal Challengers Bengaluru,"M Chinnaswamy Stadium, Bengaluru",2024-03-29,field,Kolkata Knight Riders,Kolkata Knight Riders,1
0,Away,Delhi Capitals,"Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam",2024-04-03,bat,Kolkata Knight Riders,Kolkata Knight Riders,1
6,Home,Lucknow Super Giants,"Eden Gardens, Kolkata",2024-04-14,field,Kolkata Knight Riders,Kolkata Knight Riders,1


In [77]:
team_name = "Kolkata Knight Riders"
opponent_teams = ["Royal Challengers Bangalore", "Royal Challengers Bengaluru"]  # Check for both names

# Filter matches where KKR played against either name variant of RCB
kkr_rcb_matches = mdf[
    ((mdf['team1'] == team_name) & (mdf['team2'].isin(opponent_teams))) |
    ((mdf['team1'].isin(opponent_teams)) & (mdf['team2'] == team_name))
].copy()

# Ensure the date column is in datetime format
kkr_rcb_matches['date'] = pd.to_datetime(kkr_rcb_matches['date'])

# Identify the opponent team
kkr_rcb_matches['opponent'] = kkr_rcb_matches.apply(lambda x: x['team1'] if x['team2'] == team_name else x['team2'], axis=1)

# Group by opponent, venue, date, toss winner, toss decision, match winner, result margin, and target runs
kkr_rcb_analysis = kkr_rcb_matches.groupby(
    ['opponent', 'venue', 'date', 'toss_winner', 'toss_decision', 'winner', 'result_margin', 'target_runs']
).size().reset_index(name='matches_played')

# Sort by date in ascending order
kkr_rcb_analysis = kkr_rcb_analysis.sort_values(by='date')

# Display the result
(kkr_rcb_analysis)


,opponent,venue,date,toss_winner,toss_decision,winner,result_margin,target_runs,matches_played
0,Royal Challengers Bangalore,"Eden Gardens, Kolkata",2023-04-06,Royal Challengers Bangalore,field,Kolkata Knight Riders,81.0,205.0,1
1,Royal Challengers Bangalore,"M Chinnaswamy Stadium, Bengaluru",2023-04-26,Royal Challengers Bangalore,field,Kolkata Knight Riders,21.0,201.0,1
3,Royal Challengers Bengaluru,"M Chinnaswamy Stadium, Bengaluru",2024-03-29,Kolkata Knight Riders,field,Kolkata Knight Riders,7.0,183.0,1
2,Royal Challengers Bengaluru,"Eden Gardens, Kolkata",2024-04-21,Royal Challengers Bengaluru,field,Kolkata Knight Riders,1.0,223.0,1


In [79]:
team_name = "Kolkata Knight Riders"
opponent_teams = ["Royal Challengers Bangalore", "Royal Challengers Bengaluru"]
home_venue = "Eden Gardens, Kolkata"  # KKR's home venue

# Filter matches where KKR played against either name variant of RCB
kkr_rcb_matches = mdf[
    ((mdf['team1'] == team_name) & (mdf['team2'].isin(opponent_teams))) |
    ((mdf['team1'].isin(opponent_teams)) & (mdf['team2'] == team_name))
].copy()

# Ensure the date column is in datetime format
kkr_rcb_matches['date'] = pd.to_datetime(kkr_rcb_matches['date'])

# Identify the opponent team
kkr_rcb_matches['opponent'] = kkr_rcb_matches.apply(lambda x: x['team1'] if x['team2'] == team_name else x['team2'], axis=1)

# Count total toss wins by each team
toss_win_count = kkr_rcb_matches['toss_winner'].value_counts()

# Count how many times the toss-winning team also won the match (correct decision)
toss_decision_correct = kkr_rcb_matches[kkr_rcb_matches['toss_winner'] == kkr_rcb_matches['winner']]['toss_winner'].value_counts()

# Count how many times the toss-winning team lost the match (wrong decision)
toss_decision_wrong = kkr_rcb_matches[kkr_rcb_matches['toss_winner'] != kkr_rcb_matches['winner']]['toss_winner'].value_counts()

# Count home toss wins for KKR
home_toss_wins = kkr_rcb_matches[(kkr_rcb_matches['venue'] == home_venue) & (kkr_rcb_matches['toss_winner'] == team_name)].shape[0]

# Count home match wins for KKR
home_match_wins = kkr_rcb_matches[(kkr_rcb_matches['venue'] == home_venue) & (kkr_rcb_matches['winner'] == team_name)].shape[0]

# Display results
print("Total Toss Wins:")
print(toss_win_count)
print("\nCorrect Toss Decisions (Win Toss & Win Match):")
print(toss_decision_correct)
print("\nWrong Toss Decisions (Win Toss & Lose Match):")
print(toss_decision_wrong)
print(f"\nKKR Home Toss Wins at {home_venue}: {home_toss_wins}")
print(f"KKR Home Match Wins at {home_venue}: {home_match_wins}")


Total Toss Wins:
toss_winner
Royal Challengers Bangalore    2
Kolkata Knight Riders          1
Royal Challengers Bengaluru    1
Name: count, dtype: int64

Correct Toss Decisions (Win Toss & Win Match):
toss_winner
Kolkata Knight Riders    1
Name: count, dtype: int64

Wrong Toss Decisions (Win Toss & Lose Match):
toss_winner
Royal Challengers Bangalore    2
Royal Challengers Bengaluru    1
Name: count, dtype: int64

KKR Home Toss Wins at Eden Gardens, Kolkata: 0
KKR Home Match Wins at Eden Gardens, Kolkata: 2


In [30]:
comb = comb.rename(columns={'venue_x':'venue'})
comb = comb.rename(columns={'season_x':'season'})
comb = comb.rename(columns={'striker':'batsman'})
comb["total_run"] = comb["runs_of_bat"] + comb["wide"] + comb["noballs"]
comb.to_csv("updated_deliveries.csv", index=False)
comb.insert(comb.columns.get_loc("noballs") + 1, "total_run", comb.pop("total_run"))



In [31]:
vdf = pd.DataFrame(comb.groupby(['venue'])['total_run'].sum()/(comb.groupby(['venue'])['match_id'].apply( lambda x: len(list(np.unique(x))))*2))

In [32]:
vdf.head()

,0
venue,
"Arun Jaitley Stadium, Delhi",219.900000
"Barsapara Cricket Stadium, Guwahati",144.000000
"Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow",169.000000
"Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam",193.000000
"Eden Gardens, Kolkata",191.071429


In [33]:
comb.groupby(['venue'])['total_run'].sum()

venue
Arun Jaitley Stadium, Delhi                                                       2199
Barsapara Cricket Stadium, Guwahati                                                288
Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow             2366
Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam                 772
Eden Gardens, Kolkata                                                             2675
Himachal Pradesh Cricket Association Stadium, Dharamsala                           717
M.Chinnaswamy Stadium, Bengaluru                                                  2574
MA Chidambaram Stadium, Chennai                                                   2836
Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur, Chandigarh    1638
Narendra Modi Stadium, Ahmedabad                                                  2682
Rajiv Gandhi International Stadium, Hyderabad                                     2350
Sawai Mansingh Stadium, Jaipur       

In [34]:
comb.groupby(['venue','innings'])['total_run'].sum().unstack(fill_value=0).assign(total_match_run=lambda x: x.sum(axis=1))


innings,1,2,total_match_run
venue,,,
"Arun Jaitley Stadium, Delhi",1170,1029,2199
"Barsapara Cricket Stadium, Guwahati",143,145,288
"Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow",1251,1115,2366
"Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam",450,322,772
"Eden Gardens, Kolkata",1344,1331,2675
"Himachal Pradesh Cricket Association Stadium, Dharamsala",405,312,717
"M.Chinnaswamy Stadium, Bengaluru",1355,1219,2574
"MA Chidambaram Stadium, Chennai",1495,1341,2836
"Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur, Chandigarh",819,819,1638


In [35]:
comb.groupby(['venue'])['match_id'].apply( lambda x: len(list(np.unique(x))))

venue
Arun Jaitley Stadium, Delhi                                                       5
Barsapara Cricket Stadium, Guwahati                                               1
Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow             7
Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam                2
Eden Gardens, Kolkata                                                             7
Himachal Pradesh Cricket Association Stadium, Dharamsala                          2
M.Chinnaswamy Stadium, Bengaluru                                                  7
MA Chidambaram Stadium, Chennai                                                   9
Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur, Chandigarh    5
Narendra Modi Stadium, Ahmedabad                                                  8
Rajiv Gandhi International Stadium, Hyderabad                                     6
Sawai Mansingh Stadium, Jaipur                                        

In [36]:
vdf.columns = ['avg_runs']

In [37]:
vdf.sort_values('avg_runs',ascending = False).head(20)

,avg_runs
venue,
"Arun Jaitley Stadium, Delhi",219.900000
"Rajiv Gandhi International Stadium, Hyderabad",195.833333
"Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam",193.000000
"Eden Gardens, Kolkata",191.071429
"M.Chinnaswamy Stadium, Bengaluru",183.857143
"Sawai Mansingh Stadium, Jaipur",182.400000
"Himachal Pradesh Cricket Association Stadium, Dharamsala",179.250000
"Wankhede Stadium, Mumbai",177.642857
"Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow",169.000000


In [38]:
# Grouping by 'venue' and 'innings', summing up 'total_run', and unstacking
comb_modified = comb.groupby(['venue', 'innings'])['total_run'].sum().unstack(fill_value=0)

# Calculating the number of matches for each venue
match_counts = comb.groupby('venue')['match_id'].nunique()

# Calculating average runs for Innings 1 and Innings 2
comb_modified['avg_innings_1'] = comb_modified[1] / match_counts
comb_modified['avg_innings_2'] = comb_modified[2] / match_counts

# Displaying the modified DataFrame
#print(cdf_modified)


In [39]:
vdf=comb_modified

In [40]:
vdf.head()

innings,1,2,avg_innings_1,avg_innings_2
venue,,,,
"Arun Jaitley Stadium, Delhi",1170,1029,234.000000,205.800000
"Barsapara Cricket Stadium, Guwahati",143,145,143.000000,145.000000
"Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow",1251,1115,178.714286,159.285714
"Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam",450,322,225.000000,161.000000
"Eden Gardens, Kolkata",1344,1331,192.000000,190.142857


In [41]:
vdf['avg_runs'] = vdf[['avg_innings_1', 'avg_innings_2']].mean(axis=1)  # Example to calculate average runs


In [42]:
# Calculate total match runs by summing Innings 1 and Innings 2
vdf['total_run'] = vdf[1] + vdf[2]

In [43]:
vdf.head()

innings,1,2,avg_innings_1,avg_innings_2,avg_runs,total_run
venue,,,,,,
"Arun Jaitley Stadium, Delhi",1170,1029,234.000000,205.800000,219.900000,2199
"Barsapara Cricket Stadium, Guwahati",143,145,143.000000,145.000000,144.000000,288
"Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow",1251,1115,178.714286,159.285714,169.000000,2366
"Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam",450,322,225.000000,161.000000,193.000000,772
"Eden Gardens, Kolkata",1344,1331,192.000000,190.142857,191.071429,2675


In [44]:
# Sorting,avg,total run
vdf.sort_values('avg_innings_1', ascending=False, inplace=True)

In [45]:
vdf.head(20)

innings,1,2,avg_innings_1,avg_innings_2,avg_runs,total_run
venue,,,,,,
"Arun Jaitley Stadium, Delhi",1170,1029,234.000000,205.800000,219.900000,2199
"Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam",450,322,225.000000,161.000000,193.000000,772
"Himachal Pradesh Cricket Association Stadium, Dharamsala",405,312,202.500000,156.000000,179.250000,717
"Rajiv Gandhi International Stadium, Hyderabad",1208,1142,201.333333,190.333333,195.833333,2350
"M.Chinnaswamy Stadium, Bengaluru",1355,1219,193.571429,174.142857,183.857143,2574
"Eden Gardens, Kolkata",1344,1331,192.000000,190.142857,191.071429,2675
"Wankhede Stadium, Mumbai",1294,1193,184.857143,170.428571,177.642857,2487
"Sawai Mansingh Stadium, Jaipur",923,901,184.600000,180.200000,182.400000,1824
"Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow",1251,1115,178.714286,159.285714,169.000000,2366
